# Nettoyage du Dataset Baby-foot Ynov

Ce notebook effectue le nettoyage et la préparation des données de matchs de baby-foot pour l'analyse.

## Objectifs
- Nettoyer les données brutes
- Standardiser les formats
- Créer des features pour l'analyse
- Exporter un dataset propre

## 1. Installation des dépendances

In [31]:
!pip install word2number

## 2. Import des bibliothèques

In [32]:
import pandas as pd
import numpy as np
from dateutil import parser
import re
from word2number import w2n
from datetime import datetime

# Montage de Google Drive (pour Colab)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 3. Chargement du dataset

In [33]:
FILE_PATH = '/content/drive/MyDrive/babyfoot_dataset.csv'
SEPARATEUR = ','

df = pd.read_csv(FILE_PATH, sep=SEPARATEUR)

print(df.head(5))
print(df.info())
print(f"\nDimensions du dataset : {df.shape}")

/tmp/ipython-input-2000172189.py:4: DtypeWarning: Columns (10,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(FILE_PATH, sep=SEPARATEUR)


   game_id      game_date               location table_id table_condition  \
0  G015295  Feb 06st 2023          Ynov Toulouse      T05     beer stains   
1  G023800     24-03-2023  Cafeteria (1st floor)      T07            worn   
2  G023577     2025-01-13               Gym Hall      T26       scratched   
3  G020644    Nov 11 2025      Salle Polyvalente      T21            worn   
4  G011677      30 Sep 23     Campus - Cafeteria      T26   missing screw   

      ball_type                      music_playing          referee  \
0           NaN  Spotify: Queen - We Will Rock You         Paul Kim   
1           NaN                     Indie playlist              NaN   
2           NaN  Spotify: Queen - We Will Rock You     Lena Clement   
3     mini ball                            EDM mix  Isabella Girard   
4  trainer ball                         Oldies 80s              yes   

  game_duration final_score_red  ... possession_time mood    player_comment  \
0         12.45               0

## 4. Suppression des doublons et valeurs manquantes

In [34]:
# Suppression des lignes dont les IDs critiques manquent
df.dropna(subset=['player_id', 'game_id'], inplace=True)

# Suppression des doublons
df.drop_duplicates(subset=['game_id', 'player_id'], inplace=True)

# Nettoyage global
df = df.replace('�', '', regex=True)

print(f"Après nettoyage des IDs et doublons : {df.shape[0]} lignes restantes")

Après nettoyage des IDs et doublons : 100000 lignes restantes


## 5. Nettoyage des scores

Gestion des différents formats de scores :
- Format 'x - y' dans une seule colonne
- Scores séparés dans deux colonnes
- Validation : scores entre 0 et 10

In [35]:
def clean_final_score(row):
    """Nettoie les scores red et blue quand le format 'x - y' est dans final_score_red."""
    s_red = row['final_score_red']
    s_blue = row['final_score_blue']

    if pd.isna(s_red):
        return pd.Series([pd.NA, pd.NA])

    s_red = str(s_red).strip().lower()
    s_blue = str(s_blue).strip().lower()

    # Si format 'x - y' détecté
    if ' - ' in s_red:
        parts = s_red.split(' - ')
        red = int(float(parts[0].strip()))
        blue = int(float(parts[1].strip()))
    else:
        try:
            red = int(float(s_red))
            blue = int(float(row['final_score_blue'])) if not pd.isna(row['final_score_blue']) else pd.NA
        except:
            return pd.Series([pd.NA, pd.NA])

    # Règle métier : score entre 0 et 10
    if not (0 <= red <= 10 and (blue is pd.NA or 0 <= blue <= 10)):
        return pd.Series([pd.NA, pd.NA])

    return pd.Series([red, blue])

# Application
df[['final_score_red','final_score_blue']] = df.apply(clean_final_score, axis=1).astype('Int64')

print("✓ Scores nettoyés")
print(f"Lignes avec scores manquants : {df[df['final_score_red'].isna()].shape[0]}")

✓ Scores nettoyés
Lignes avec scores manquants : 0


## 6. Standardisation des couleurs d'équipe et du gagnant

In [36]:
def standardize_color(color):
    """Standardise les couleurs d'équipe en 'RED' ou 'BLUE'."""
    if pd.isna(color):
        return 'UNKNOWN'

    color = str(color).strip().lower()

    if color in ['red', 'r', 'redteam', '🔴', 'rouge']:
        return 'RED'
    elif color in ['blue', 'b', 'blueteam','🔵', 'bleu']:
        return 'BLUE'
    elif color in ['tie', 'draw']:
        return 'DRAW'
    else:
        return 'UNKNOWN'

# Application
print("Valeurs uniques avant standardisation :")
print(f"team_color: {df['team_color'].unique()}")
print(f"winner: {df['winner'].unique()}")

df['winner'] = df['winner'].apply(standardize_color)
df['team_color'] = df['team_color'].apply(standardize_color)

# Suppression des matchs inutilisables
df = df[df['winner'] != 'UNKNOWN']

print(f"\n✓ Couleurs standardisées")
print(f"Lignes restantes : {df.shape[0]}")

Valeurs uniques avant standardisation :
team_color: ['Red' 'Blue' 'R' 'B' '🔴' '🔵' 'blue' 'red']
winner: ['Blue' 'red' 'Bleu' 'blue' 'Red' nan 'R' 'B' 'BLUE' 'Rouge' 'draw' 'TIE'
 'RED' 'tie']

✓ Couleurs standardisées
Lignes restantes : 95340


## 7. Nettoyage des statistiques joueurs

Traitement des buts, assists, saves (conversion texte → nombre)

In [37]:
def clean_player_stat(stat):
    """Nettoie les statistiques de joueur (buts, saves, etc.)"""
    if pd.isna(stat):
        return 0

    stat = str(stat).strip().lower()

    # Gérer les cas textuels (ex: 'two' -> 2)
    text_to_num = {
        'zero': 0, 'one': 1, 'two': 2,
        'three': 3, 'four': 4, 'five': 5
    }
    if stat in text_to_num:
        return text_to_num[stat]

    try:
        return int(float(stat))
    except:
        return 0

stats_cols = ['player_goals', 'player_own_goals', 'player_assists', 'player_saves']
for col in stats_cols:
    df[col] = df[col].apply(clean_player_stat)

print("✓ Statistiques joueurs nettoyées")
print("\nAperçu des statistiques :")
print(df[stats_cols].describe())

✓ Statistiques joueurs nettoyées

Aperçu des statistiques :
       player_goals  player_own_goals  player_assists  player_saves
count  95340.000000      95340.000000    95340.000000  95340.000000
mean       2.484959          0.029285        2.503891      4.996769
std        2.486927          0.218080        1.709463      3.162448
min        0.000000          0.000000        0.000000      0.000000
25%        0.000000          0.000000        1.000000      2.000000
50%        2.000000          0.000000        3.000000      5.000000
75%        4.000000          0.000000        4.000000      8.000000
max       10.000000          2.000000        5.000000     10.000000


## 8. Conversion des dates

Parsing des dates multilingues (FR/EN) + extraction du jour de la semaine

In [38]:
# Dictionnaire pour traduire les mois français vers anglais
mois_fr = {
    "janv": "jan", "févr": "feb", "fevr": "feb", "mars": "mar", "avr": "apr",
    "mai": "may", "juin": "jun", "juil": "jul", "août": "aug", "aout": "aug",
    "sept": "sep", "oct": "oct", "nov": "nov", "déc": "dec", "dec": "dec"
}

def nettoyer_date(date_str):
    if pd.isna(date_str):
        return None

    s = date_str.strip().lower()

    # Supprimer suffixes anglais : 1st, 2nd, 3rd, 4th...
    s = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', s)

    # Traduire les mois français → anglais
    for fr, en in mois_fr.items():
        s = re.sub(fr, en, s, flags=re.IGNORECASE)

    # Uniformiser les séparateurs
    s = re.sub(r'[-/.]', ' ', s)

    try:
        if re.match(r'^\s*\d{4}\s', s):
            parsed = parser.parse(s, yearfirst=True, dayfirst=False)
        else:
            parsed = parser.parse(s, dayfirst=True)
        return parsed
    except Exception:
        return None

df['game_date'] = df['game_date'].apply(nettoyer_date)
df['DayOfWeek'] = df['game_date'].dt.day_name()

print("✓ Dates converties")
print("\nAperçu des dates :")
print(df[['game_date', 'DayOfWeek']].head(10))

✓ Dates converties

Aperçu des dates :
    game_date DayOfWeek
0  2023-02-06    Monday
1  2023-03-24    Friday
2  2025-01-13    Monday
3  2025-11-11   Tuesday
4  2023-09-30  Saturday
5  2023-07-04   Tuesday
6  2023-11-16  Thursday
8  2023-03-20    Monday
9  2024-02-04    Sunday
10 2023-06-24  Saturday


## 9. Conversion de la durée

Conversion de différents formats de durée en secondes :
- HH:MM:SS ou MM:SS
- "10min" ou "4 min"
- Format décimal "4.1" ou "4,1"

In [39]:
def duration_to_seconds(duration):
    """Convertit différentes représentations de durée en secondes."""
    if pd.isna(duration):
        return np.nan

    s = str(duration).strip().lower()

    # Format HH:MM:SS ou MM:SS
    if ':' in s:
        try:
            parts = list(map(int, s.split(':')))
            if len(parts) == 3:
                return parts[0]*3600 + parts[1]*60 + parts[2]
            elif len(parts) == 2:
                return parts[0]*60 + parts[1]
        except:
            return np.nan

    # Format "10min", "4 min"
    match_min = re.match(r'(\d+(\.\d+)?|\d+,\d+)\s*min', s)
    if match_min:
        val = match_min.group(1).replace(',', '.')
        try:
            return int(float(val) * 60)
        except:
            return np.nan

    # Format décimal en minutes "4,1" ou "4.1"
    match_decimal = re.match(r'^\d+([.,]\d+)?$', s)
    if match_decimal:
        val = s.replace(',', '.')
        try:
            return int(float(val) * 60)
        except:
            return np.nan

    return np.nan

df['game_duration'] = df['game_duration'].apply(duration_to_seconds)

print("✓ Durées converties en secondes")
print(f"Durée moyenne : {df['game_duration'].mean():.0f} secondes ({df['game_duration'].mean()/60:.1f} minutes)")

✓ Durées converties en secondes
Durée moyenne : 1050 secondes (17.5 minutes)


## 10. Nettoyage du nombre de participants

In [40]:
def clean_attendance(value):
    """Extrait le nombre entier du nombre de joueurs."""
    if pd.isna(value):
        return np.nan
    s = str(value).strip().lower()

    # Cherche le premier nombre dans la chaîne
    match = re.search(r'\d+', s)
    if match:
        return int(match.group())
    return np.nan

# Étape 1 : nettoyer attendance_count
df["attendance_count"] = df["attendance_count"].apply(clean_attendance).astype("Int64")

# Étape 2 : calculer le nombre de joueurs réels par match
df["real_player_count"] = df.groupby("game_id")["player_id"].transform("nunique")

# Étape 3 : corriger attendance_count si NaN ou inférieur au nombre de joueurs
df["attendance_count"] = df.apply(
    lambda row: row["real_player_count"]
    if pd.isna(row["attendance_count"]) or row["attendance_count"] < row["real_player_count"]
    else row["attendance_count"],
    axis=1
)

# supprimer la colonne temporaire
df = df.drop(columns="real_player_count")

print("✓ Nombre de participants nettoyé")
print(f"Nombre moyen de participants : {df['attendance_count'].mean():.1f}")

✓ Nombre de participants nettoyé
Nombre moyen de participants : 5.1


## 11. Nettoyage des âges

In [41]:
def nettoyer_age(age_str):
    if pd.isna(age_str):
        return None

    s = str(age_str).lower().strip()

    # Supprimer les caractères non alphanumériques (sauf espaces)
    s = re.sub(r'[^\w\s]', '', s)

    # Supprimer les mots inutiles
    s = re.sub(r'\b(yrs?|years?|ans?|yo)\b', '', s)
    s = s.strip()

    # Si la chaîne contient un nombre → extraire le premier
    num_match = re.search(r'\d+', s)
    if num_match:
        return int(num_match.group())

    # Sinon, tenter de convertir les mots anglais en nombre
    try:
        return w2n.word_to_num(s)
    except Exception:
        return None

df["player_age"] = df["player_age"].apply(nettoyer_age).astype('Int64')

print("✓ Âges nettoyés")
print(f"Âge moyen : {df['player_age'].mean():.1f} ans")

✓ Âges nettoyés
Âge moyen : 23.8 ans


## 12. Nettoyage des autres colonnes

Standardisation de :
- Location
- Player Role
- Referee
- Season

In [42]:
# Location
df['location'] = (df['location']
    .str.strip()
    .str.upper()
    .replace(['TOULOUSE','YNOV TLS', 'YNOV - BÂTIMENT A','YNOV_TLS'], 'YNOV TOULOUSE')
    .fillna('UNKNOWN'))

# Player Role
df['player_role'] = (df['player_role']
    .str.strip()
    .str.upper()
    .replace(['DEF','DEFENCE'], 'DEFENSE')
    .replace(['ATT', 'ATTCK'], 'ATTACK')
    .fillna('UNKNOWN'))

# Referee
df['referee'] = (df['referee']
    .str.strip()
    .str.upper()
    .replace(['YES','NO', 'PLAYER1'], 'NONE')
    .fillna('NONE'))

# Season
df['season'] = (df['season']
    .str.strip()
    .replace(['s24/25','Season 24-25'], '2024/2025')
    .replace('2025 Season', '2025'))

print("✓ Colonnes standardisées")
print(f"\nLocations : {df['location'].unique()}")
print(f"Rôles : {df['player_role'].unique()}")
print(f"Saisons : {df['season'].unique()}")

✓ Colonnes standardisées

Locations : ['YNOV TOULOUSE' 'CAFETERIA (1ST FLOOR)' 'GYM HALL' 'SALLE POLYVALENTE'
 'CAMPUS - CAFETERIA' 'BAR LE FOOS' 'STUDENT HOUSE' 'LAB 204']
Rôles : ['DEFENSE' 'ATTACK']
Saisons : ['2023/2024' '2024/2025' '2025']


## 13. Standardisation des ratings

Conversion des ratings en échelle numérique (1-5)

In [43]:
def standardize_rating(rating):
    """Convertit le rating en une échelle numérique (1-5)."""
    if pd.isna(rating):
        return np.nan

    rating = str(rating).strip().lower()

    # Mapping des symboles et mots-clés
    if '⭐⭐⭐⭐⭐' in rating or 'five' in rating:
        return 5
    elif '⭐⭐⭐⭐' in rating or 'four' in rating or '🙂' in rating:
        return 4
    elif '⭐⭐⭐' in rating or 'three' in rating or '😐' in rating:
        return 3
    elif '⭐⭐' in rating or 'two' in rating:
        return 2
    elif '⭐' in rating or 'one' in rating or '😡' in rating:
        return 1

    # Conversion simple pour les notes numériques directes
    try:
        val = float(rating)
        return val if 1 <= val <= 5 else np.nan
    except:
        return np.nan

df['rating_raw'] = df['rating_raw'].apply(standardize_rating).astype('Int64')

print("✓ Ratings standardisés")

✓ Ratings standardisés


## 14. Aperçu du dataset nettoyé

In [44]:
print("=" * 70)
print("APERÇU DU DATASET NETTOYÉ")
print("=" * 70)

print(f"\nDimensions : {df.shape[0]} lignes × {df.shape[1]} colonnes")
print(f"\nNombre de matchs uniques : {df['game_id'].nunique()}")
print(f"\nNombre de joueurs uniques : {df['player_id'].nunique()}")
print(f"\nPériode couverte : {df['game_date'].min()} → {df['game_date'].max()}")

print("\n" + "=" * 70)
df.head()

APERÇU DU DATASET NETTOYÉ

Dimensions : 95340 lignes × 36 colonnes

Nombre de matchs uniques : 23835

Nombre de joueurs uniques : 800

Période couverte : 2023-01-01 00:00:00 → 2025-12-31 00:00:00



,game_id,game_date,location,table_id,table_condition,ball_type,music_playing,referee,game_duration,final_score_red,...,mood,player_comment,team_color,is_substitute,ping_ms,notes,duplicate_flag,misc,created_at,DayOfWeek
0,G015295,2023-02-06,YNOV TOULOUSE,T05,beer stains,NaN,Spotify: Queen - We Will Rock You,PAUL KIM,747,0,...,1,ref biased,RED,yes,185,injured,NaN,NaN,2025-10-02T10:41:54,Monday
1,G023800,2023-03-24,CAFETERIA (1ST FLOOR),T07,worn,NaN,Indie playlist,NONE,514,10,...,🙂,NaN,BLUE,yes,NaN,NaN,0,-,2025-10-02T10:41:55,Friday
2,G023577,2025-01-13,GYM HALL,T26,scratched,NaN,Spotify: Queen - We Will Rock You,LENA CLEMENT,1032,2,...,2,NaN,RED,no,NaN,double booked,0,NaN,2025-10-02T10:41:55,Monday
3,G020644,2025-11-11,SALLE POLYVALENTE,T21,worn,mini ball,EDM mix,ISABELLA GIRARD,310,6,...,3,team spirit high,BLUE,maybe,NaN,double booked,NaN,-,2025-10-02T10:41:54,Tuesday
4,G011677,2023-09-30,CAMPUS - CAFETERIA,T26,missing screw,trainer ball,Oldies 80s,NONE,360,3,...,😂,rage quit,RED,yes,NaN,NaN,NaN,NaN,2025-10-02T10:41:54,Saturday


## 15. Exportation du dataset nettoyé

In [45]:
# Sélection des colonnes finales
colonnes_finales = [
    'game_id', 'game_date', 'DayOfWeek', 'location', 'game_duration',
    'final_score_red', 'final_score_blue', 'winner',
    'player_id', 'player_canonical_name', 'team_color', 'player_role',
    'player_goals', 'player_own_goals', 'player_assists', 'player_saves',
    'referee', 'attendance_count', 'season', 'recorded_by', 'rating_raw',
    'player_comment', 'player_age'
]

# Création du dataset final
df_clean = df.filter(items=colonnes_finales).copy()

# Enregistrement du fichier
df_clean.to_csv('dataset_Ynov_babyfoot_CLEAN.csv', index=False)

print("NETTOYAGE TERMINÉ")
print(f"\nLignes finales : {df_clean.shape[0]}")
print(f"\nFichier exporté : dataset_Ynov_babyfoot_CLEAN.csv")

NETTOYAGE TERMINÉ

Lignes finales : 95340

Fichier exporté : dataset_Ynov_babyfoot_CLEAN.csv


## 📝 Résumé des transformations

### Transformations effectuées :

1. **Doublons et valeurs manquantes** : Suppression des lignes avec IDs critiques manquants
2. **Scores** : Normalisation du format 'x - y' et validation (0-10)
3. **Couleurs** : Standardisation RED/BLUE/DRAW
4. **Statistiques joueurs** : Conversion texte → nombre (buts, assists, saves)
5. **Dates** : Parsing multilingue FR/EN + extraction du jour de la semaine
6. **Durées** : Conversion en secondes (formats HH:MM:SS, "Xmin", décimal)
7. **Participants** : Extraction du nombre de joueurs
8. **Âges** : Nettoyage et conversion en nombre
9. **Localisation** : Standardisation des lieux
10. **Rôles** : Uniformisation ATTACK/DEFENSE
11. **Arbitres** : Normalisation
12. **Saisons** : Standardisation du format
13. **Ratings** : Conversion en échelle 1-5

###  Dataset prêt pour l'analyse et la modélisation IA !

Le fichier `dataset_Ynov_babyfoot_CLEAN.csv` peut maintenant être utilisé pour :
- Analyse exploratoire des données (EDA)
- Visualisations statistiques
- Modélisation prédictive
- Analyse des performances joueurs
- Détection de patterns de jeu